In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:

import sys
sys.path.insert(0, '..')
from odynn.circuit import Circuit
from odynn.models import LeakyIntegrate, ChemSyn, GapJunction
import numpy as np
import torch

inputs = ['AVBL', 'AVBR', 'VA12', 'PVCR', 'PVCL', 'DVA']

DT = 1
N_parallel = 1

def get_neur(dt = DT, par=N_parallel):
    n = LeakyIntegrate.create_default(1, par, tensors=True, dt=dt)
    return n


In [ ]:
import data
import pylab as plt
import torch
import seaborn as sns

N_sets = 4
plt.figure(figsize=(30,8))
traces = np.zeros((4126, N_sets, len(inputs)))
dt = torch.zeros(N_sets,1,1)
for i,j in enumerate([0,1,2,4]):
    allt = data.get_data(j)
    allt['PVCR'] = np.zeros(allt.shape[0])
    allt['PVCL'] = np.zeros(allt.shape[0])
    traces[:allt.shape[0],i] = allt[inputs]
    dt[i] = 4000 / allt.shape[0]
    # sns.heatmap(traces.corr(), cmap='bwr', vmin=-1, vmax=1, annot=True)
    plt.subplot(N_sets,1,i+1)
    plt.plot(traces[:,i])

plt.show()
print(dt)

In [ ]:
from tqdm import tqdm

target = torch.Tensor(traces)[:,None,:,:,None]
# target = torch.Tensor(np.repeat(traces.values[:400, :], 5, axis=0))[:,None,None,:,None]

# ts = np.arange(0,400,1)
# target = np.full(ts.shape, -60) + 20 * 100/(ts+1) * ((ts > 30) & (ts < 70)) + 100 * ((ts > 49) & (ts < 51)) + 130 * ((ts > 85) & (ts < 86))
# target = torch.Tensor(target)[:,None,None,None,None]
print(target.shape)
# plt.plot()
init = target[0]

inj = torch.zeros((target.shape[0], N_sets, len(inputs), 1))
inj.requires_grad = True

neuron = get_neur(dt=dt)
losses = []
# Define loss function
loss_fn = torch.nn.MSELoss(reduction='sum')
learning_rate = 0.05
optimizer = torch.optim.Adam([inj], lr=learning_rate)


for t in tqdm(range(200)):
    y = neuron.calculate(inj, init)
    # Compute MSE
    print(y.shape, target.shape)
    loss = loss_fn(y, target)
    losses.append(loss.item())

    print(loss.item())
    #Upgrade variables
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if t%1 == 0:
        for i in range(N_sets):
            plt.subplot(311)
            plt.plot(target[:,0,i,:,0].detach().numpy())
            plt.subplot(312)
            plt.plot(y[:,0,i,:,0].detach().numpy())
            plt.subplot(313)
            plt.plot(inj[:,i,:,0].detach().numpy())
            plt.show()
            plt.close()



In [ ]:
import pickle

with open('currents', 'wb') as f:
    pickle.dump([inputs, inj.detach().numpy(), dt, neuron._param], f)

In [ ]:
injtest = torch.zeros(400,1,10,1)
injtest[:200] = 3

neuron.dt = 1.

neuron._param['C_m'] = 5

y = neuron.calculate(injtest, init)

plt.subplot(211)
plt.plot(y[:,0,0,:,0].detach().numpy())
plt.subplot(212)
plt.plot(injtest[:,0,:,0].detach().numpy())
plt.show()
plt.close()